In [ ]:
import pandas as pd
import json
import re
import ast
import pickle
import glob
from time import process_time, sleep
import numpy as np
import torch
from torch_geometric.data import InMemoryDataset, download_url
import networkx as nx
from networkx.algorithms import community
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from pyvis.network import Network
import string
import colorama
from colorama import Fore
import termtables as tt
string.punctuation
import openai
import time
import os
import seaborn as sns
import itertools

In [ ]:
df=pd.read_csv('./Data/Insight_label_pred.csv',index_col=0)
df

In [ ]:
Insight_dataset=pd.read_csv('./Data/Insight_dataset.csv',index_col=0)
Insight_dataset

# Add summarization to insight dataset 

In [ ]:
for i in range(len(Insight_dataset)):
    s_solved=[]
    s_unsolved=[]
    for j in range(len(df)):
        if Insight_dataset['corpusid'][i] == df['corpusid'][j] and df['Label'][j] == 'ReSolved':
            s_solved.append(df['Sentence'][j])
        if Insight_dataset['corpusid'][i] == df['corpusid'][j] and df['Label'][j] == 'Finding':
            s_unsolved.append(df['Sentence'][j])
    Insight_dataset['ReSolved'][i] = ''.join(s_solved)
    Insight_dataset['Finding'][i] = ''.join(s_unsolved)
            

In [ ]:
Insight_dataset

# Citation net

In [ ]:
# columns = ['sourceNodeId','targetNodeId','cite_info','cite_text','relationshipType','flag']
columns = ['sourceNodeId','targetNodeId','cite_info','relationshipType','flag']
d_edges=pd.DataFrame(columns=columns)

In [ ]:
count=0
for i in range(len(Insight_dataset)):
    try:
        List=eval(Insight_dataset['ref_paper_id'][i])
        # cite_dic=eval(Insight_dataset['cite_text'][i])
        count=count+1
    except:
        continue
    for j in List:
        if j in list(Insight_dataset['corpusid']):
            # d_edges_append=pd.DataFrame(data=[[Insight_dataset['corpusid'][i],j,str(List),cite_dic[j].replace('\n',''),'cites',count]], columns=columns)
            d_edges_append=pd.DataFrame(data=[[Insight_dataset['corpusid'][i],j,str(List),'cites',count]], columns=columns)
            d_edges=pd.concat([d_edges, d_edges_append], ignore_index=True, axis=0)

In [ ]:
d_edges

In [ ]:
d_edges.to_csv('./Data/d_edges.csv')

In [ ]:
total_id=list(set(d_edges['sourceNodeId']))
total_id.extend(list(set(d_edges['targetNodeId'])))

In [ ]:
print(len(set(total_id)))
total_id=list(set(total_id))

In [ ]:
#id -> index -> content
re_num={}
for i in range (len(total_id)):
    content={}
    content['num']=i
    content['type']=1
    for j in range(len(Insight_dataset)):
        if Insight_dataset['corpusid'][j]==total_id[i]:
            # content['author-date']=total[total_id[i]]
            content['title']=Insight_dataset['paper-title'][j]
            content['pdfurl']=Insight_dataset['pdfurl'][j]
            content['conclusion']=Insight_dataset['Content'][j]
            content['ReSolved']=Insight_dataset['ReSolved'][j]
            content['Finding']=Insight_dataset['Finding'][j]

            if 'a survey' in content['title'] or 'A survey' in content['title'] or 'a Survey' in content['title'] or 'A Survey' in content['title']:
                content['type']=0
            else:
                content['type']=1
            
            # print('true')
            break
        
    re_num[total_id[i]]=content
re_num

In [ ]:
list_source=[]
list_target=[]
for i in range(len(d_edges)):
    if d_edges['sourceNodeId'][i]==d_edges['targetNodeId'][i]:
        continue
   
    list_source.append(re_num[d_edges['sourceNodeId'][i]]['num'])
    list_target.append(re_num[d_edges['targetNodeId'][i]]['num'])

In [ ]:
class Demo_Data(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    #file
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
   
    @property
    def processed_file_names(self):
        return ['data.pt']
    # download from website
    # def download(self):
    #     # Download to `self.raw_dir`.
    #     download_url(url, self.raw_dir)
        ...
    #data generate
    def process(self):
        # Read data into huge `Data` list.
        # Read data into huge `Data` list.
        # building
        
        start=0
        end=len(list_source)
        
        Edge_index = torch.tensor([list_source,
                                   list_target], dtype=torch.long)
        
        # s=list_source[start:end].copy()
        # s.extend(list_target[start:end])
        print(len(re_num))
        
        # node_num=len(set(s))

        # node feature
        sample_feature=[0]*len(re_num)
        X = torch.tensor(np.array(sample_feature).reshape(-1, 1).tolist(), dtype=torch.float)
        # node label
        # Y = torch.tensor([0,1,0],dtype=torch.float)
        
        #building
        # data = Data(x=X, edge_index=Edge_index, y=Y)
        data = Data(x=X,edge_index=Edge_index)
        
        # put in datalist
        data_list = [data]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        data.num_nodes = len(re_num)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
dataset = Demo_Data("Whole_Network")
data=dataset[0]
dataset.get_summary()

In [ ]:
G = to_networkx(data, to_undirected=False)
degrees = [val for (node, val) in G.degree()]
display(pd.DataFrame(pd.Series(degrees).describe()).transpose().round(2))
print(len(degrees))
print(sum(degrees))
plt.figure(figsize=(10, 6))
# plt.hist(degrees, bins=100)
# plt.hist(degrees, range(0,10))
plt.hist(degrees, range(0,50),rwidth=0.5,align="left")

plt.xlabel("node degree")
plt.show()

In [ ]:
ancestors={}
ancestors_num=[]
for i in G.nodes:
    # print(i)
    # print(nx.ancestors(G, i))
    ancestors[i]=len(nx.ancestors(G, i))
    ancestors_num.append(len(nx.ancestors(G, i)))

In [ ]:
# degrees = [val for (node, val) in G.degree()]
i=1
r_node_list=[]
while i<=10: 
    print('rank:',i)
    r_node=ancestors_num.index(sorted(ancestors_num,reverse=True)[0])
    print('high_contribution_node_num:',r_node)
    r_node_list.append(r_node)
    print('number of cited:',len(nx.ancestors(G, r_node)))
    print('cited_by:',nx.ancestors(G, r_node))
    G.nodes[r_node]['color']='deeppink'
    G.nodes[r_node]['shape']='triangle'
    G.nodes[r_node]['size']=35
    ancestors_num[r_node]=0 # clear
    i=i+1

In [ ]:
#make graph - total
word_node_num=len(total_id)
for i in re_num:
    #paper node shape,word,edge
    if re_num[i]['type']==0:
        G.nodes[re_num[i]['num']]['shape']='star'
        G.nodes[re_num[i]['num']]['color']='chocolate'
        # G.nodes[re_num[i]['num']]['label']='S'+str(re_num[i]['num'])
        G.nodes[re_num[i]['num']]['size']=50

    #paper label title
    try:
        # G.nodes[re_num[i]['num']]['label']=re_num[i]['author-date']
        G.nodes[re_num[i]['num']]['title']=re_num[i]['title']+'\n'+re_num[i]['pdfurl']+'\n'+str(re_num[i]['num'])
    except:
        continue

    word_node_num=word_node_num+4

net = Network(notebook=True, cdn_resources='in_line',directed=True,bgcolor='#fffefe')
net.from_nx(G)
net.toggle_physics(False)
net.show_buttons()
net.show('./Output/Cite_net_total.html')

# Similarity

In [ ]:
#nltk_word
def word_div(strings):
    strings=strings.lower()
    strings=strings.translate(str.maketrans('', '', string.punctuation))
    token=nltk.word_tokenize(strings)
    stop_words = stopwords.words('english')
    tokens=[]
    for i_2 in range(len(token)):
        if token[i_2] not in stop_words:
            tokens.append(token[i_2])   
    filter_word=['from', 'subject', 're', 'edu', 'use','introduction','conclusion','evaluation','figure','table','survey','et','al','research','we','use','work','issue','also','future','however','finally','task','tasks','paper','nlp','recent','natural','language','processing','nevertheless']
    clean_token=[]
    for item in tokens:
        item=lemmatizer.lemmatize(item, pos="n")
        if item not in filter_word and re.search(r'^\d+$',item) is None:
            # clean_token.append(st.stem(item))
            clean_token.append(item)
    return clean_token

In [ ]:
#Similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer, util
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
lemmatizer = WordNetLemmatizer()
st = LancasterStemmer()
model = SentenceTransformer('allenai/scibert_scivocab_uncased')
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
sentences=[]
strings_total=''
branch=2
pair_list=[]
pair_list_total=[]
for item in re_num:
    if 'conclusion' not in re_num[item].keys():
        # print(re_num[item]['num'])
        continue
    
    sentences.append(re_num[item]['conclusion'])
    strings_total=strings_total+re_num[item]['conclusion']
    


#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i_1 in range(len(cosine_scores)-1):
    for j_1 in range(i_1+1, len(cosine_scores)):
        pairs.append({'index': [i_1, j_1], 'score': cosine_scores[i_1][j_1]})

#Sort scores in decreasing order
pairs_dec = sorted(pairs, key=lambda x: x['score'], reverse=True)

#Simliarity of each article (n-branch)
start=0
for i in range(1,len(pairs)):
    strings_pair=''
    flag=pairs[i-1]['index'][0]
    i_1, j_1 = pairs[i]['index']
    if i_1!=flag:
        pair=pairs[start:i]
        pair = sorted(pair, key=lambda x: x['score'], reverse=True)
        # for building tree
        for j in range(branch):
            pair[j]['s1']=sentences[pair[j]['index'][0]]
            pair[j]['s2']=sentences[pair[j]['index'][1]]
            strings_pair=sentences[pair[j]['index'][0]]+sentences[pair[j]['index'][1]]
            # print(Fore.BLACK + "{} \t\t {} \t\t Score: {:.4f}".format(sentences[pair[j]['index'][0]], sentences[pair[j]['index'][1]], pair[j]['score']))

            #nltk->上位語,top:10
            clean_token=word_div(strings_pair)
            freq_dist = nltk.FreqDist(clean_token)
            # print(Fore.BLUE + 'hypernym:',freq_dist.most_common(10))
            pair[j]['hypernym']=freq_dist.most_common(10)

            #nltk ->共通語
            S_0=word_div(sentences[pair[j]['index'][0]])
            S_1=word_div(sentences[pair[j]['index'][1]])
            and_list=set(S_0)&set(S_1)
            # print(Fore.MAGENTA + 'common_word:',and_list)
            pair[j]['common_word']=and_list

            #add data to list
            pair_list.append(pair[j])

        start=i


In [ ]:
for j in range(len(pairs)):
    pairs[j]['s1']=sentences[pairs[j]['index'][0]]
    pairs[j]['s2']=sentences[pairs[j]['index'][1]]
    strings_pair=sentences[pairs[j]['index'][0]]+sentences[pairs[j]['index'][1]]
    # print(Fore.BLACK + "{} \t\t {} \t\t Score: {:.4f}".format(sentences[pair[j]['index'][0]], sentences[pair[j]['index'][1]], pair[j]['score']))

    #nltk->Hypernym,top:10
    clean_token=word_div(strings_pair)
    freq_dist = nltk.FreqDist(clean_token)
    # print(Fore.BLUE + 'hypernym:',freq_dist.most_common(10))
    pairs[j]['hypernym']=freq_dist.most_common(10)

    #Common_word
    S_0=word_div(sentences[pairs[j]['index'][0]])
    S_1=word_div(sentences[pairs[j]['index'][1]])
    and_list=set(S_0)&set(S_1)
    # print(Fore.MAGENTA + 'common_word:',and_list)
    pairs[j]['common_word']=and_list


In [ ]:
pairs

In [ ]:
pair_list

# Subgraph - Inheritance tree

In [ ]:
#Copy
G_ext=G.copy()

In [ ]:
#Number of citations in whole net
ancestors={}
ancestors_num=[]
for i in G_ext.nodes:
    # print(i)
    # print(nx.ancestors(G_ext, i))
    ancestors[i]=len(nx.ancestors(G_ext, i))
    ancestors_num.append(len(nx.ancestors(G_ext, i)))

ancestors_org=ancestors_num.copy()

In [ ]:
#node-select-tree
root_max=10 # init 
layer_max=5
leaf_lim=2 # number of leaf
tree_list=[] # node be selected
node_his=[]
for n in range(root_max):# Topn cited-root
     # history
    root_list=[] # roor list   
    print(f'path{n}:')
    path_dict={}
    ancestors_num=ancestors_org.copy()
    i=1 #layer_num
    # print('rank:',i)
    for m in range(len(G_ext.nodes)):
        if ancestors_num.index(sorted(ancestors_num,reverse=True)[m]) not in node_his:
            root=ancestors_num.index(sorted(ancestors_num,reverse=True)[m])
            print('root:',root)
            break
    
    G_ext.nodes[root]['color']='deeppink'
    G_ext.nodes[root]['shape']='star'
    ancestors_num[root]=0 
    node_his.append(root)
    root_list.append(root)

    while True:
        if root_list==[]:
            break
        # if ancestors_num.index(sorted(ancestors_num,reverse=True)[0])==0:
        #     break
        print('Current_node:',root_list[0])
        print('number of cited:',len(nx.ancestors(G_ext, root_list[0])))
        print('cited_by:',nx.ancestors(G_ext, root_list[0]))
        # G_ext.nodes[root]['label']=f'layer{i}'
        
    
        for l in range(leaf_lim):
            count=0
            while count<len(G):
                candidate=ancestors_num.index(sorted(ancestors_num,reverse=True)[0])
                if candidate in list(nx.ancestors(G_ext, root_list[0])) and candidate not in node_his:
                    i=i+1
                    path_dict.setdefault(root_list[0],[]).append(candidate)
                    print('candidate:',candidate)
                    root_list.append(candidate)
                    node_his.append(candidate)
                    break
                ancestors_num[candidate]=0 
                count=count+1
        
        root_list.pop(0)
        print('root_list:',root_list)
                
    tree_list.append(path_dict)

In [ ]:
tree_list

In [ ]:
strings_list=[]
for i in range(len(tree_list)):
    strings=''
    nodes=list(set(tree_list[i]))
    for node in nodes:
        for item in list(re_num.values()):
            if node == item['num']:
                key = [k for k, v in re_num.items() if v['num'] == node][0]
                break
        strings=strings+re_num[key]['conclusion']
    
    strings_list.append(strings)
strings_list
        

In [ ]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
from nltk import stem
def imp_word(d1,d2):
    words_list=[]
    document1=tb(d1)
    document2=tb(d2)
    bloblist = [document1, document2]

    stemmer = stem.LancasterStemmer()
    stop_words = stopwords.words('english')
    
    for i, blob in enumerate(bloblist):
        # print("Top words in document {}".format(i + 1))
        scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        for word, score in sorted_words[:7]:
            if word in stop_words:
                continue

            clean_word=stemmer.stem(word)
            # print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
            words_list.append(str(word))
        break

    return words_list
    

In [ ]:
color_list = ['red',
           'goldenrod',
           'springgreen',
           'limegreen',
           'green',
           'salmon',
           'violet',
           'pink',
           'sienna',
           'palegreen',
           'darkcyan',
           'orchid',
           'magenta',
           'slategrey',
           'olive',
           'indigo',
           'blueviolet',
           'gold',
           'aquamarine',
           'lightcoral',
           'teal',
           'fuchsia',
           '#03f5dd']

In [ ]:
# build-graph : Tree structure
inv=100
inv_title=35
# color_list=['Pink','Red','Blue','Yellow','Brown','Pink','Green']
G_generate= Network(notebook=True, 
cdn_resources='in_line',
directed=True,
height='600px',
width='100%',
font_color="blue",
heading='Inheritance tree'
)
history=[]
for i in range(len(tree_list)):
    color=color_list[i]
    flag=0
    for j in tree_list[i]:
        for item in list(re_num.values()):
            if j == item['num']:
                key_j = [k for k, v in re_num.items() if v['num'] == j][0]
                words_list=imp_word(re_num[key_j]['conclusion'],strings_list[i])
                try:
                    G_generate.add_node(j, color=color,label='\n'.join([re_num[key_j]['title'][i:i+inv_title] for i in range(0, len(re_num[key_j]['title']),inv_title)]),
                    title='- Keyword: '+', '.join(words_list)+'\n\n'+
                '- ReSolved: '+'\n'.join([re_num[key_j]['ReSolved'][i:i+inv] for i in range(0, len(re_num[key_j]['ReSolved']), inv)])+'\n\n'+
                '- Finding: '+'\n'.join([re_num[key_j]['Finding'][i:i+inv] for i in range(0, len(re_num[key_j]['Finding']), inv)])+'\n\n'+'- Url: '+re_num[key_j]['pdfurl'],
                shape='box',
                size=60,
                group=i)
                except:
                    G_generate.add_node(j, color=color,label=re_num[key_j]['title'],title='None',group=i)
            
        if flag==0:
            G_generate.nodes[G_generate.node_ids.index(j)]['shape']='ellipse'
            flag=1
            
        for z in tree_list[i][j]:
            key_z = [k for k, v in re_num.items() if v['num'] == z][0]
            words_list=imp_word(re_num[key_z]['conclusion'],strings_list[i])
            try:
                G_generate.add_node(z, color=color,label='\n'.join([re_num[key_z]['title'][i:i+inv_title] for i in range(0, len(re_num[key_z]['title']),inv_title)]),
                title='- Keyword: '+', '.join(words_list)+'\n\n'+
                '- ReSolved: '+'\n'.join([re_num[key_z]['ReSolved'][i:i+inv] for i in range(0, len(re_num[key_z]['ReSolved']), inv)])+'\n\n'+
                '- Finding: '+'\n'.join([re_num[key_z]['Finding'][i:i+inv] for i in range(0, len(re_num[key_z]['Finding']), inv)])+'\n\n'+'- Url: '+re_num[key_z]['pdfurl'],
                shape='box',
                size=40,
                group=i)
            except:
                G_generate.add_node(z, color=color,label=str(z),title='None',group=i)

            if [j, z] in history:
                continue

            # add edge(thickness,length)
            flag=0
            ## find corpus id
          
            for c in range(len(d_edges)):
                if d_edges['targetNodeId'][c] == key_j and d_edges['sourceNodeId'][c] == key_z:
                    # cite_t = d_edges['cite_text'][c]
                    break

            for p in pairs:
                if p['index']==[j,z] or p['index']==[z,j]:
                    G_generate.add_edge(j, z,
                    gravity=-8000, 
                    color=color,
                    label='\n\n'.join(list(p['common_word'])[0:10]),
                    title='Similarity: '+str(round(float(p['score']),3)),
                    spring_length=10000,
                    width=float(p['score'])*5,
                    spring_strength=0.001,
                    overlap=1,
                    )
                    flag=1
                    break
            # if flag==0:
            #     G_generate.add_edge(j, z, color=color,width=0.1)

            # G_generate.add_edge(j, z,
            # label='edge',
            # gravity=-10,
            # central_gravity=0.3,
            # spring_length=z,
            # spring_strength=0.001,
            # overlap=1)
            
            history.append([j,z])
     

G_generate.bgcolor="#ffffee"
G_generate.barnes_hut()

options = """const options = {
  "layout": {
    "hierarchical": {
      "enabled": true,
      "levelSeparation": 320,
      "nodeSpacing": 155,
      "sortMethod": "directed"
    }
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0,
      "nodeDistance": 180,
      "damping": 0.46,
      "avoidOverlap": 1
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  }
}
"""

G_generate.set_options(options)
# G_generate.set_edge_smooth('dynamic')

# G_generate.show_buttons(filter_=['layout','physics'])
# G_generate.show_buttons()
G_generate.show('./Output/Inheritance_tree.html',notebook=False)

In [ ]:
# sub-net
sub_net=[]
for i in range(len(node_his)):
    sub_net.append(node_his[i])
    # sub_net.extend([n for n in G_ext.neighbors(cluster_list[i])])

#Sub-Grpah
# nx.draw(G.subgraph(sub_net),with_labels=True)
net = Network(notebook=True, cdn_resources='in_line',directed=True,bgcolor='#fffefe')
net.from_nx(G_ext.subgraph(sub_net))
net.show_buttons()
net.show('./Output/Subnet_paper_in_inheritance_tree.html')

# Subgraph - Relevance-tree

In [ ]:
FW_list=[]
Solved_list=[]
sentences_fw=[]
sentences_solved=[]
for key in re_num:
    if re_num[key]['Finding']!='':
        FW_list.append(re_num[key]['num'])
        sentences_fw.append(re_num[key]['Finding'])
    if re_num[key]['ReSolved']!='':
        Solved_list.append(re_num[key]['num'])
        sentences_solved.append(re_num[key]['ReSolved'])

#Compute embeddings
embeddings_FW = model.encode(sentences_fw, convert_to_tensor=True)
embeddings_Solved = model.encode(sentences_solved, convert_to_tensor=True)
cosine_scores_novelty = util.cos_sim(embeddings_FW, embeddings_Solved)

In [ ]:
s_c=20

avg_list=[]
max_list_index=[]

for i in range(len(cosine_scores_novelty)):
    l=[]
    for z in range(s_c):
        l.append(Solved_list[int(np.argsort(cosine_scores_novelty[i])[-(z+s_c)])])

    avg_list.append(sum(cosine_scores_novelty[i])/len(cosine_scores_novelty[i]))
    max_list_index.append(l)

avg_list_index_org=list(np.argsort(avg_list)[::-1])


In [ ]:
#adjustment
avg_list_index=[]
for i in range(len(avg_list_index_org)):
    avg_list_index.append(FW_list[avg_list_index_org[i]])


In [ ]:
#node-select-tree
root_max=10 # init 
layer_max=5
leaf_lim=2 # number of leaf
tree_list=[] # node be selected
node_his=[]
for n in range(root_max):# Topn cited-root
     # history
    root_list=[] # root list   
    print(f'path{n}:')
    path_dict={}
    i=1
    avg_list=avg_list_index.copy()
    # print('rank:',i)
    root=int(avg_list_index[n])
    print(root)
    
    G_ext.nodes[root]['color']='deeppink'
    G_ext.nodes[root]['shape']='star'
    node_his.append(root)
    root_list.append(root)

    while True:
        if i>layer_max:
            break
        if root_list==[]:
            break
        # if ancestors_num.index(sorted(ancestors_num,reverse=True)[0])==0:
        #     break
        print('Current_node:',root_list[0])
        # G_ext.nodes[root]['label']=f'layer{i}'
        
    
        for l in range(leaf_lim):
            for s in range(s_c):
                candidate=max_list_index[FW_list.index(root)][s]
                if candidate not in node_his:
                    path_dict.setdefault(root_list[0],[]).append(candidate)
                    print('candidate:',candidate)
                    root_list.append(candidate)
                    node_his.append(candidate)
                    break

        i=i+1
        root_list.pop(0)
        print('root_list:',root_list)
                
    tree_list.append(path_dict)

In [ ]:
tree_list

In [ ]:
strings_list=[]
for i in range(len(tree_list)):
    strings=''
    nodes=list(set(tree_list[i]))
    for node in nodes:
        for item in list(re_num.values()):
            if node == item['num']:
                key = [k for k, v in re_num.items() if v['num'] == node][0]
                break
        strings=strings+re_num[key]['conclusion']
    
    strings_list.append(strings)
strings_list

In [ ]:
# build-graph : Tree structure
inv=100
inv_title=35
# color_list=['Pink','Red','Blue','Yellow','Brown','Pink','Green']
G_Novelty= Network(notebook=True, 
cdn_resources='in_line',
directed=True,
height='600px',
width='100%',
font_color="blue",
heading='Relevance-tree'
)
history=[]
for i in range(len(tree_list)):
    color=color_list[i]
    flag=0
    for j in tree_list[i]:
        for item in list(re_num.values()):
            if j == item['num']:
                key_j = [k for k, v in re_num.items() if v['num'] == j][0]
                words_list=imp_word(re_num[key_j]['conclusion'],strings_list[i])
                try:
                    G_Novelty.add_node(j, color=color,label='\n'.join([re_num[key_j]['title'][i:i+inv_title] for i in range(0, len(re_num[key_j]['title']),inv_title)]),
                    title='- ReSolved: '+'\n'.join([re_num[key_j]['ReSolved'][i:i+inv] for i in range(0, len(re_num[key_j]['ReSolved']), inv)])+'\n\n'+
                    '- Finding: '+'\n'.join([re_num[key_j]['Finding'][i:i+inv] for i in range(0, len(re_num[key_j]['Finding']), inv)])+'\n\n'+'- Url: '+re_num[key_j]['pdfurl'],
                    shape='box',
                    size=60,
                    group=i)
                except:
                    G_Novelty.add_node(j, color=color,label=str(j),title='None',group=i)
            
        if flag==0:
            # G_Novelty.nodes[G_Novelty.node_ids.index(j)]['shape']='ellipse'
            flag=1
            
        for z in tree_list[i][j]:
            key_z = [k for k, v in re_num.items() if v['num'] == z][0]
            words_list=imp_word(re_num[key_z]['conclusion'],strings_list[i])
            try:
                G_Novelty.add_node(z, color=color,label='\n'.join([re_num[key_z]['title'][i:i+inv_title] for i in range(0, len(re_num[key_z]['title']),inv_title)]),
                title='- Keyword: '+', '.join(words_list)+'\n\n'+
                '- ReSolved: '+'\n'.join([re_num[key_z]['ReSolved'][i:i+inv] for i in range(0, len(re_num[key_z]['ReSolved']), inv)])+'\n\n'+
                '- Finding: '+'\n'.join([re_num[key_z]['Finding'][i:i+inv] for i in range(0, len(re_num[key_z]['Finding']), inv)])+'\n\n'+'- Url: '+re_num[key_z]['pdfurl'],
                shape='box',
                size=40,
                group=i)
            except:
                G_Novelty.add_node(z, color=color,label=str(z),title='None',group=i)

            if [j, z] in history:
                continue

            # add edge(thickness,length)
            flag=0
            ## find corpus id
            cite_r='False'
            for c in range(len(d_edges)):
                if d_edges['targetNodeId'][c] == key_j and d_edges['sourceNodeId'][c] == key_z:
                    cite_r='True'
                    break
                if d_edges['sourceNodeId'][c] == key_j and d_edges['targetNodeId'][c] == key_z:
                    cite_r='True'
                    break
              

            for p in pairs:
                if p['index']==[j,z] or p['index']==[z,j]:
                    G_Novelty.add_edge(j, z,
                    gravity=-8000, 
                    color=color,
                    label='\n\n'.join(list(p['common_word'])[0:10]),
                    title='Similarity: '+str(round(float(p['score']),3))+'\n\n'+'Cite_relationship: '+cite_r,
                    spring_length=10000,
                    width=float(p['score'])*5,
                    spring_strength=0.001,
                    overlap=1,
                    )
                    flag=1
                    break
            # if flag==0:
            #     G_generate.add_edge(j, z, color=color,width=0.1)

            # G_generate.add_edge(j, z,
            # label='edge',
            # gravity=-10,
            # central_gravity=0.3,
            # spring_length=z,
            # spring_strength=0.001,
            # overlap=1)
            
            history.append([j,z])
     

G_Novelty.bgcolor="#ffffee"
G_Novelty.barnes_hut()

options = """const options = {
  "layout": {
    "hierarchical": {
      "enabled": true,
      "levelSeparation": 320,
      "nodeSpacing": 155,
      "sortMethod": "directed"
    }
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0,
      "nodeDistance": 180,
      "damping": 0.46,
      "avoidOverlap": 1
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  }
}
"""

G_Novelty.set_options(options)
# G_generate.set_edge_smooth('dynamic')

# G_generate.show_buttons(filter_=['layout','physics'])
# G_generate.show_buttons()
G_Novelty.show('./Output/Relevance_tree.html',notebook=False)